In [2]:
import json
import pyspark
import numpy as np
import pandas as pd
from py2neo import *
from pymongo import MongoClient
# from scipy.spatial import distance
from pyspark.sql import SparkSession
from sklearn.metrics.pairwise import cosine_similarity
from biobert_embedding.embedding import BiobertEmbedding

### BioBERT Embeddings

In [3]:
text = "Breast cancers with HER2 amplification have a higher risk of CNS metastasis and poorer prognosis."

In [4]:
biobert = BiobertEmbedding()
word_embeddings = biobert.word_vector(text)
sentence_embedding = biobert.sentence_vector(text)

In [5]:
print("Text Tokens: ", biobert.tokens)

Text Tokens:  ['breast', 'cancers', 'with', 'her2', 'amplification', 'have', 'a', 'higher', 'risk', 'of', 'cns', 'metastasis', 'and', 'poorer', 'prognosis', '.']


In [6]:
print ('Shape of Word Embeddings: %d x %d' % (len(word_embeddings), len(word_embeddings[0])))

Shape of Word Embeddings: 16 x 768


In [7]:
type(word_embeddings[0])

torch.Tensor

In [8]:
print("Shape of Sentence Embedding = ", len(sentence_embedding))

Shape of Sentence Embedding =  768


### Data Preparation 

In [9]:
data = pd.read_csv('../../Data/ClientData/testdata.csv')

data.drop(index = 0, inplace = True)
data.reset_index(inplace = True)
data.drop(['index'], axis = 1, inplace = True)

print(f'Total Number of Entries: {len(data)}')
data.head(2)

Total Number of Entries: 48


Item number  Supplier Code Supplier Catalog Number OM Product Number  \
0       2007.0         4304.0                  57769C        430457769C   
1       2008.0         4304.0                   61105        4304061105   

   Replacement Number                              Supplier Product Desc  \
0                 NaN                       Tray Laceration --High Point   
1                 NaN  Suture Removal Tray (Sharp-Blunt Sc./Insert Fo...   

                    Short Description1                   Short Description2  \
0  Tray Laceration --High Point                                         NaN   
1  Tray Suture Removal Sharp Blunt      Comp  Medc 1314SUT1002                

   Category Code Class Code  ... Pallet Quantity UPC  \
0          510.0     91-397  ...             NaN NaN   
1          511.0     91-770  ...             NaN NaN   

   Shipment Destination Code Layer Quantity  Shelf Life(Days) DOT (Ground)  \
0                          D            NaN               NaN          NaN   
1                          D            NaN               NaN          NaN   

  IATA (Air)  IMDG (Water)  NMFC Code  Freight Class  
0        NaN           NaN        NaN            NaN  
1        NaN           NaN        NaN            NaN  

[2 rows x 88 columns]

In [10]:
data = data[['Supplier Product Desc',
             'Brand',
             'Operative Noun',
             'Material',
             'Manufacturer',
             'Item number']]

# type(data['Material'][0])
data.replace(np.nan, 'Not available', inplace = True)
data['Item number'] = np.int64(data['Item number'])
data['Item number'] = data['Item number'].map(str)
data.head(2)

Supplier Product Desc  \
0                       Tray Laceration --High Point   
1  Suture Removal Tray (Sharp-Blunt Sc./Insert Fo...   

                       Brand Operative Noun       Material  \
0  Medical Action Industries           TRAY  Not available   
1  Medical Action Industries         SUTURE  Not available   

                    Manufacturer Item number  
0  Medical Action Industries Inc        2007  
1  Medical Action Industries Inc        2008

### BioBERT Sentence Embeddings for Selected Features

In [11]:
biobert = BiobertEmbedding()
embeddings = {}

for index in range(len(data)):

    string = data['Supplier Product Desc'][index] + ' ' + \
             data['Brand'][index] + ' ' + \
             data['Operative Noun'][index] + ' ' + \
             data['Material'][index] + ' ' + \
             data['Manufacturer'][index]
       
#     print (string, '\n')

#     a = pipe.predict(s)
#     embeddings[i] = np.mean(a['biobert_embeddings'])
    
    sentence_embedding = biobert.sentence_vector(string)
    embeddings[index] = sentence_embedding

### Cosine Similarity Calculation

In [12]:
similar_products = []

for index in range(len(data)):
    for second_index in range(len(data)):
        
        if index != second_index:
            
            something = [data['Item number'][index],
                         data['Manufacturer'][index],
                         data['Supplier Product Desc'][index],
                         data['Item number'][second_index],
                         data['Supplier Product Desc'][second_index],
#                          1 - distance.cosine(embeddings[index], embeddings[second_index]),
                         cosine_similarity(embeddings[index].reshape(1, -1), embeddings[second_index].reshape(1, -1))[0][0]]
            
            similar_products.append(something)

In [13]:
similar_products = pd.DataFrame(columns = ['source', 
                                           'src_manufacturer',
                                           'src_desc', 
                                           'dest', 
                                           'dest_desc',                                            
                                           'cosine_similarity'],
                                data = similar_products)
similar_products.head(2)

source               src_manufacturer                      src_desc  dest  \
0   2007  Medical Action Industries Inc  Tray Laceration --High Point  2008   
1   2007  Medical Action Industries Inc  Tray Laceration --High Point  2009   

                                           dest_desc  cosine_similarity  
0  Suture Removal Tray (Sharp-Blunt Sc./Insert Fo...           0.940730  
1  Tray General-Purpose Procedure With Curved Mos...           0.902727

### Top 5 Similar Products for each Product

In [14]:
final_similar_products_list = []

for unique_product_id in similar_products['source'].unique():
#     print(unique_product_id)
    temp_df = similar_products.query(f'source == "{unique_product_id}"')
    temp_df = temp_df.sort_values(by = ['cosine_similarity'], ascending = [False]).head(5)
    
    temp_df.reset_index(inplace = True)
    temp_df.drop(['index'], axis = 1, inplace = True)    
#     print(temp_df.head())
    top_similar_products_list = []
    
    for index, product_data in temp_df.iterrows():       
        
        if (index == 0):
            top_similar_products_list.append([product_data['src_manufacturer'],
                                              product_data['source'],
                                              product_data['src_desc'],
                                              product_data['dest'],
                                              product_data['dest_desc'],
                                              round(product_data['cosine_similarity'] * 100, 2)])
        else:
            top_similar_products_list.append([product_data['dest'],
                                              product_data['dest_desc'],
                                              round(product_data['cosine_similarity'] * 100, 2)])
    
    final_similar_products_list.append([val for sublist in top_similar_products_list for val in sublist])    
    
#     break

In [15]:
final_df = pd.DataFrame(data = final_similar_products_list, 
                        columns = ['Manufacturer', 'Product_ID', 'Product_Description',
                                   'First_Match_ID', 'First_Match_Description', 'First_Match_Score',
                                   'Second_Match_ID', 'Second_Match_Description', 'Second_Match_Score',
                                   'Third_Match_ID', 'Third_Match_Description', 'Third_Match_Score',
                                   'Fourth_Match_ID', 'Fourth_Match_Description', 'Fourth_Match_Score',
                                   'Fifth_Match_ID', 'Fifth_Match_Description', 'Fifth_Match_Score'])
final_df    

Manufacturer Product_ID  \
0   Medical Action Industries Inc       2007   
1   Medical Action Industries Inc       2008   
2   Medical Action Industries Inc       2009   
3                       Novaplus®       2010   
4                       Novaplus®       2011   
5                       Novaplus®       2012   
6                       Novaplus®       2013   
7                       Novaplus®       2014   
8                       Novaplus®       2015   
9                       Novaplus®       2016   
10                      Novaplus®       2017   
11                      Novaplus®       2018   
12  Medical Action Industries Inc       2019   
13  Medical Action Industries Inc       2020   
14  Medical Action Industries Inc       2021   
15                  Not available       2022   
16  Medical Action Industries Inc       2023   
17  Medical Action Industries Inc       2024   
18  Medical Action Industries Inc       2025   
19                  Not available       2026   
20  Medical Action Industries Inc       2027   
21  Medical Action Industries Inc       2028   
22  Medical Action Industries Inc       2029   
23  Medical Action Industries Inc       2030   
24  Medical Action Industries Inc       2031   
25  Medical Action Industries Inc       2032   
26  Medical Action Industries Inc       2033   
27  Medical Action Industries Inc       2034   
28  Medical Action Industries Inc       2035   
29  Medical Action Industries Inc       2036   
30  Medical Action Industries Inc       2037   
31  Medical Action Industries Inc       2038   
32  Medical Action Industries Inc       2039   
33  Medical Action Industries Inc       2040   
34  Medical Action Industries Inc       2041   
35  Medical Action Industries Inc       2042   
36  Medical Action Industries Inc       2043   
37  Medical Action Industries Inc       2044   
38                      Novaplus®       2045   
39                      Novaplus®       2046   
40                      Novaplus®       2047   
41                      Novaplus®       2048   
42                      Novaplus®       2049   
43                      Novaplus®       2050   
44                      Novaplus®       2051   
45                      Novaplus®       2052   
46                      Novaplus®       2053   
47                      Novaplus®       2054   

                                  Product_Description First_Match_ID  \
0                        Tray Laceration --High Point           2033   
1   Suture Removal Tray (Sharp-Blunt Sc./Insert Fo...           2036   
2   Tray General-Purpose Procedure With Curved Mos...           2013   
3   Minor Procedure Tray With Stainless Steel Inst...           2011   
4   Laceration Tray With Stainless Steel Instruments            2012   
5   Laceration Tray With Stainless Steel Instrumen...           2011   
6   Kit Suture Removal With Plastic Instrument 4In...           2046   
7   Reel Sterilization Heat-Seal Clear White 100Fe...           2033   
8                             Tray Laceration Ochsner           2053   
9          Foam Strip Needle Counters 60 Count Foam             2017   
10          Foam Strip Needle Counters 30 Count Foam            2016   
11        Foam Strip Needle Counters Magnet 30 Count            2017   
12                           Roll Trayliner 12 X 80             2030   
13  Kit Iv Start Chlorhexidine-Gluconate Sepp Appl...           2043   
14                    Stick Sponge 1 3/4 Jumbo 5/Gls            2044   
15                             Surgical Light Shield            2014   
16                Cleaner Cautery Tip 2X2Inch Sterile           2025   
17                  Kit Action Pac Surgical Southwest           2019   
18  Sponge Laparotomy Pre-Washed White 4X18Inch St...           2029   
19  Tubular Bandage Applicators  A Variety Of Stai...           2011   
20                 Needle Counter Foam Strip Adhesive           2038   
21  Sponge Laparotomy Pre-Washed With Ring White 1...           2029   
22  Sponge L

### Save DataFrame to MongoDB

In [16]:
# final_df.to_csv('BioBERT_Similarity_Result.csv')

In [17]:
client = MongoClient("mongodb://127.0.0.1:27017")
db = client.MED_EX_CHAIN
product_similarity = db.PRODUCT_SIMILARITY

In [18]:
product_similarity.insert_many(json.loads(final_df.to_json(orient = 'records')))

### Save Data From MongoDB TO Neo4J

In [19]:
# Connecting to Neo4J
graph = Graph("bolt://localhost:7687", auth = ("neo4j", "neo4j"))

In [20]:
# Connecting PySprak to MongoDB
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        config("spark.executor.memory", "1g").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

In [23]:
# Reading Data from MongoDB
df_product_similarity = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT_SIMILARITY").load()

In [24]:
df_product_similarity.show()

+-----------------------+--------------+-----------------+-----------------------+--------------+-----------------+------------------------+---------------+------------------+--------------------+--------------------+----------+------------------------+---------------+------------------+-----------------------+--------------+-----------------+--------------------+
|Fifth_Match_Description|Fifth_Match_ID|Fifth_Match_Score|First_Match_Description|First_Match_ID|First_Match_Score|Fourth_Match_Description|Fourth_Match_ID|Fourth_Match_Score|        Manufacturer| Product_Description|Product_ID|Second_Match_Description|Second_Match_ID|Second_Match_Score|Third_Match_Description|Third_Match_ID|Third_Match_Score|                 _id|
+-----------------------+--------------+-----------------+-----------------------+--------------+-----------------+------------------------+---------------+------------------+--------------------+--------------------+----------+------------------------+-------------

In [25]:
# Creating Product-Similarity Node
for record in df_product_similarity.rdd.collect():
    graph.run("merge (n: Product_Similarity{manufacturer_name: $manufacturer_name, \
                                            product_id: $product_id, \
                                            product_description: $product_description, \
                                            first_match_id: $first_match_id, \
                                            first_match_description: $first_match_description, \
                                            first_match_score: $first_match_score, \
                                            second_match_id: $second_match_id, \
                                            second_match_description: $second_match_description, \
                                            second_match_score: $second_match_score, \
                                            third_match_id: $third_match_id, \
                                            third_match_description: $third_match_description, \
                                            third_match_score: $third_match_score, \
                                            fourth_match_id: $fourth_match_id, \
                                            fourth_match_description: $fourth_match_description, \
                                            fourth_match_score: $fourth_match_score, \
                                            fifth_match_id: $fifth_match_id, \
                                            fifth_match_description: $fifth_match_description, \
                                            fifth_match_score: $fifth_match_score})",
                manufacturer_name = record['Manufacturer'],
                product_id = record['Product_ID'],
                product_description = record['Product_Description'],
                first_match_id = record['First_Match_ID'],
                first_match_description = record['First_Match_Description'],
                first_match_score = record['First_Match_Score'],
                second_match_id = record['Second_Match_ID'],
                second_match_description = record['Second_Match_Description'],
                second_match_score = record['Second_Match_Score'],
                third_match_id = record['Third_Match_ID'],
                third_match_description = record['Third_Match_Description'],
                third_match_score = record['Third_Match_Score'],
                fourth_match_id = record['Fourth_Match_ID'],
                fourth_match_description = record['Fourth_Match_Description'],
                fourth_match_score = record['Fourth_Match_Score'],
                fifth_match_id = record['Fifth_Match_ID'],
                fifth_match_description = record['Fifth_Match_Description'],
                fifth_match_score = record['Fifth_Match_Score'])
